#Trabajo Final Grupo Nº 2

Rafael Perez	rafaelperezctes@gmail.com
Olinca Ayala	olincayalan88@gmail.com
###@Created on Dec Sat 2 16:57:20 2023
###@author: Grupo Nº 2


# BD Integrsas e integronas y mas de Shewanella

Este cuaderno realiza la descarga y análisis de secuencias nucleotídicas de integronas de Shewanella utilizando Biopython.


In [8]:
# @title Instalación de librerias
import subprocess
import ipywidgets as widgets
from IPython.display import display

def instalar_librerias(b=""):
    with output:
        librerias = [
            "import_ipynb",
            "nbconvert",
            "ipywidgets",  # Asegúrate de que ya esté instalado
            "biopython",
            "pandas",
            "numpy",
            "matplotlib",
            "tqdm",
            "odfpy",
            "requests",
            "gdown"
        ]

        barra_progreso = widgets.IntProgress(
            value=0,
            min=0,
            max=len(librerias),
            bar_style='info',
            orientation='horizontal'
        )

        etiqueta = widgets.Label(value='Iniciando instalación...')
        display(widgets.VBox([etiqueta, barra_progreso]))

        for libreria in librerias:
            etiqueta.value = f"Instalando: {libreria}..."
            subprocess.run(["pip", "install", libreria])
            barra_progreso.value += 1

        etiqueta.value = 'Instalación completada.'

# Crear botón
boton_instalar = widgets.Button(description='Instalar Librerías')

# Crear área de salida
output = widgets.Output()
display(output)
#instalar_librerias()

Output()

In [ ]:
#@title 2) descargar cvs con id

import subprocess
from tqdm.notebook import tqdm

def descargar_archivos_desde_google_drive(ids_archivos):
    for id_archivo in tqdm(ids_archivos, desc="Descargando archivos"):
        url = f"https://drive.google.com/uc?id={id_archivo}"
        comando_descarga = ["gdown", "--id", id_archivo]
        proceso = subprocess.Popen(comando_descarga, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        stdout, stderr = proceso.communicate()

        if proceso.returncode != 0:
            print(f"Error al descargar el archivo con ID {id_archivo}: {stderr.decode().strip()}")
        else:
            print(f"Archivo con ID {id_archivo} descargado exitosamente.")

#falla excel "1B4smxRJe77roNY-b3u7lMjdKd7vM_dREp3_3O_UdtmU"   # integrones_shewanella
# Lista de IDs de archivos de Google Drive
ids_archivos = [
    "1mkYUcDlFZLVSc2tTJMgOV7xA4PvegO4_",  # Integrasas_de_Integron_Shew.csv
    "16RdgLIn4GANuLmoayAI20QEt086SqBzh",  # Integrasas_de_Integron.csv
    "1fHxLBiSnpXPXULMxnuGumJFgM46ZVlIh",  # Integrasas_de_Integron.ods
    "1M_JRAG74APAUTVXiZ-z9iPtXjQ2Md__3"   # integrones_shewanella
]

# Llamar a la función
#descargar_archivos_desde_google_drive(ids_archivos)

In [ ]:
import pandas as pd

def leer_y_mostrar_archivo(ruta_archivo, tipo_archivo='csv', separador='\t'):
    if tipo_archivo == 'csv':
        df = pd.read_csv(ruta_archivo, sep=separador)
    elif tipo_archivo == 'ods':
        df = pd.read_excel(ruta_archivo, engine='odf')
    else:
        raise ValueError("Tipo de archivo no soportado")

    print(df)
    print("Columnas:", df.columns.tolist())
    print("\n #################################################### \n")

# Ejemplos de uso:
#leer_y_mostrar_archivo('Integrasas_de_Integron.csv', 'csv')
#leer_y_mostrar_archivo('Integrasas_de_Integron_Shew.csv', 'csv')
#leer_y_mostrar_archivo('Integrasas_de_Integron.ods', 'ods')


In [ ]:
import re
def clean_protein_ids(proteins):
    clean_proteins = []
    for protein in proteins:
        clean_protein = re.sub(r'[^A-Za-z0-9.]', '', protein)
        clean_proteins.append(clean_protein)
    return clean_proteins
#protein_ids = df['ID protein'].tolist()
#clean_ids = clean_protein_ids(protein_ids)

In [ ]:
#@title 2) descargar y guardar_secuencias_nucleotidicas

def descargar_y_guardar_secuencias_nucleotidicas(ncbi_ids):
    from Bio import Entrez, SeqIO
    from Bio.SeqRecord import SeqRecord
    from Bio.Seq import Seq
    from urllib.error import HTTPError
    Entrez.email = "tu_email@example.com"  # Tu email real aquí
    secuencias = []

    for ncbi_id in ncbi_ids:
        try:
            handle = Entrez.efetch(db="nucleotide", id=ncbi_id, rettype="fasta", retmode="text")
            seq_record = SeqIO.read(handle, "fasta")
            handle.close()
            secuencias.append(seq_record)

            # Guardar la secuencia en un archivo FASTA
            fasta_filename = f"{ncbi_id}.fasta"
            SeqIO.write(seq_record, fasta_filename, "fasta")
            print("Se ha descargado: ",fasta_filename)
        except HTTPError as e:
            print(f"Error al descargar la secuencia para el ID {ncbi_id}: {e}")
    print()
    return secuencias

In [ ]:
#@title 2) descargar y guardar_secuencias_proteinas

# Función para descargar secuencias de proteínas desde NCBI y guardarlas en archivos FASTA
def descargar_y_guardar_secuencias_proteinas(ncbi_ids):
    import pandas as pd
    from Bio import Entrez, SeqIO
    from Bio.Entrez import HTTPError
    Entrez.email = "tu_email@example.com"  # Sustituye esto con tu email real
    secuencias = []

    for ncbi_id in ncbi_ids:
        try:
            handle = Entrez.efetch(db="protein", id=ncbi_id, rettype="fasta", retmode="text")
            seq_record = SeqIO.read(handle, "fasta")
            handle.close()
            secuencias.append(seq_record)

            # Guardar la secuencia en un archivo FASTA
            fasta_filename = f"{ncbi_id}.fasta"
            SeqIO.write(seq_record, fasta_filename, "fasta")
            print("Se ha descargado: ",fasta_filename)
        except HTTPError as e:
            print(f"Error al descargar la secuencia para el ID {ncbi_id}: {e}")
    print()
    return secuencias

In [ ]:

def main():
    # Funciones auxiliares previamente definidas
    instalar_librerias()
    import pandas as pd
    descargar_archivos_desde_google_drive(ids_archivos)

    # Descargar y analizar secuencias nucleotídicas
    df_nucleotidicas = pd.read_csv('Integrasas_de_Integron.csv', sep='\t')
    secuencias_nucleotidicas = descargar_y_guardar_secuencias_nucleotidicas(df_nucleotidicas['# GenBank '].tolist())

    # Descargar y analizar secuencias de proteínas (CSV)
    df_proteinas_csv = pd.read_csv('Integrasas_de_Integron_Shew.csv', sep='\t')
    protein_ids_csv = df_proteinas_csv['PROTEIN_ID'].tolist()
    clean_ids_csv = clean_protein_ids(protein_ids_csv)
    secuencias_proteinas_csv = descargar_y_guardar_secuencias_proteinas(clean_ids_csv)

    # Descargar y analizar secuencias de proteínas (ODS)
    file_path_ods = 'Integrasas_de_Integron.ods'
    df_proteinas_ods = pd.read_excel(file_path_ods, engine='odf')
    protein_ids_ods = df_proteinas_ods['ID protein'].tolist()
    clean_ids_ods = clean_protein_ids(protein_ids_ods)
    secuencias_proteinas_ods = descargar_y_guardar_secuencias_proteinas(clean_ids_ods)

# Ejecutar la función main
#main()



# teoria, codigo, pruebas, etc.

notebook1:

var = 5

def greeting():
  print("Hola!")


main:

import notebook1
print(notebook1.var)
notebook1.greeting()


----------------------------
Excelente pregunta.

Para visualizar un gráfico Matplotlib desde otro notebook que importa la función, hay un par de opciones:

1. Devuelve el objeto Figure desde la función:

```python
# notebook1.py
import matplotlib.pyplot as plt

def grafica():
   fig = plt.figure()
   plt.plot([1,2,3])
   return fig
```

```python
# notebook2.py

import notebook1

fig = notebook1.grafica()
fig # Se muestra el gráfico
```

2. Utiliza `matplotlib inline`:

```python
# notebook1.py

import matplotlib.pyplot as plt
plt.switch_backend('inline')

def grafica():
   plt.figure()
   plt.plot([1,2,3])
   plt.show()   
```

```python
# notebook2.py

import notebook1
notebook1.grafica() # Se muestra el gráfico
```

La opción 2 es más simple, `inline` hace que la imagen se muestre automáticamente.

Espero que esto te ayude a compartir visualizaciones entre notebooks de Colab. ¡Cualquier otra duda me comentas!
